In [ ]:
from Bio.PDB import PDBParser
import numpy as np
import random

def load_coordinates(pdb_file):
    """Load coordinates from a PDB file.
    Returns a numpy array of shape (n_atoms, 3) for all CA atoms."""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('PDB', pdb_file)
    coords = []
    for model in structure:
        for chain in model: #iterate over chains
            for residue in chain: #iterate over residues
                if residue.has_id('CA'):  # Only consider alpha carbons
                    coords.append(residue['CA'].get_coord()) #extract xyz
    return np.array(coords)

def crmsd(coords1, coords2):
    # Center the coordinates
    c1_centered = coords1 - coords1.mean(axis=0)
    c2_centered = coords2 - coords2.mean(axis=0)
    
    # Compute optimal rotation using Kabsch algorithm
    H = c1_centered.T @ c2_centered
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    
    # Apply rotation to coords1
    c1_rot = c1_centered @ R
    
    # Compute RMSD
    diff = c1_rot - c2_centered
    return np.sqrt(np.mean(np.sum(diff**2, axis=1)))


def pairwise_distance(coords):
    """Compute pairwise distance matrix for a set of coordinates.
    Returns (n x n) where i and j is the distance between i and j"""
    n = len(coords)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            dist =np.linalg.norm(coords[i] - coords[j]) # Euclidean distance
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist # Symmetric matrix
    return dist_matrix

def drmsd(D1, D2, indices=None):
    if indices is None:
        # Use all unique upper-triangle pairs (i < j)
        ixs = [(i, j) for i in range(len(D1)) for j in range(i+1, len(D1))]
    else:
        ixs = indices # Use provided indices
    diff_sq = [(D1[i, j] - D2[i, j])**2 for i, j in ixs] #calculate squared diff for each pair
    return np.sqrt(np.mean(diff_sq))


In [36]:
coords1 = load_coordinates("6lu7_ca_50_99.pdb")
coords2 = load_coordinates("2amd_ca_50_99.pdb")

In [37]:
crmsd = crmsd(coords1, coords2)
print(f"c-RMSD: {crmsd:.3f} Å")

c-RMSD: 0.794 Å


In [38]:
D1 = pairwise_distance(coords1)
D2 = pairwise_distance(coords2)
drmsd_all = drmsd(D1, D2)
print(f"(b) d-RMSD (all pairs): {drmsd_all:.3f} Å")

(b) d-RMSD (all pairs): 0.264 Å


In [ ]:
#generate ca-ca pairs, sort them and use 600 shortest pairs
ixs_all = [(i, j) for i in range(len(D1)) for j in range(i+1, len(D1))]
sorted_ixs = sorted(ixs_all, key=lambda ij: (D1[ij[0], ij[1]] + D2[ij[0], ij[1]]) / 2)
drmsd_600_shortest = drmsd(D1, D2, indices=sorted_ixs[:600])
print(f"(c) d-RMSD (600 shortest pairs): {drmsd_600_shortest:.3f} Å")


(c) d-RMSD (600 shortest pairs): 0.257 Å


In [ ]:
#use random sampling to get 600 pairs
random_ixs = random.sample(ixs_all, 600)
drmsd_600_random = drmsd(D1, D2, indices=random_ixs)
print(f"(d) d-RMSD (600 random pairs): {drmsd_600_random:.3f} Å")

(d) d-RMSD (600 random pairs): 0.263 Å
